In [1]:
CREATE OR REPLACE TEMPORARY VIEW dns
USING org.apache.spark.sql.parquet
OPTIONS (
  path "s3a://data/dns/"
)

Loading spark

Exception in thread Thread-3:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.10/site-packages/sparkmonitor/kernelextension.py", line 126, in run
    self.onrecv(msg)
  File "/opt/conda/lib/python3.10/site-packages/sparkmonitor/kernelextension.py", line 143, in onrecv
    sendToFrontEnd({
  File "/opt/conda/lib/python3.10/site-packages/sparkmonitor/kernelextension.py", line 223, in sendToFrontEnd
    monitor.send(msg)
  File "/opt/conda/lib/python3.10/site-packages/sparkmonitor/kernelextension.py", line 57, in send
    self.comm.send(msg)
AttributeError: 'ScalaMonitor' object has no attribute 'comm'


""


In [2]:
CREATE OR REPLACE TEMPORARY VIEW dns_history
USING org.apache.spark.sql.parquet
OPTIONS (
  path "s3a://test/temp/dns_history/"
)

""


In [3]:
CREATE OR REPLACE TEMPORARY VIEW dns_report
AS
SELECT count(DISTINCT user) users, count(*) resolutions, date
FROM dns
WHERE date >= date_add(current_date(), -3)
GROUP BY date

""


In [4]:
INSERT OVERWRITE DIRECTORY "s3a://test/temp/dir1"
USING parquet
SELECT * FROM dns_history
WHERE date < date_add(current_date(), -3)

""


In [5]:
CREATE OR REPLACE TEMPORARY VIEW dir1
USING org.apache.spark.sql.parquet
OPTIONS (
  path "s3a://test/temp/dir1"
)

""


In [6]:
INSERT OVERWRITE DIRECTORY "s3a://test/temp/dns_history/"
USING parquet
SELECT * FROM dns_report
UNION ALL
SELECT * FROM dir1

""


In [22]:
SELECT "${env:S3_ACCESS_KEY}" AS ENV, 1


,ENV,1
0,zDdnekZrHIyltouc,1
